In [117]:
from hashlib import sha256
import json
from ecdsa import SigningKey, VerifyingKey, SECP256k1
from datetime import datetime
import pickle

class Transaction:
    def __init__(self, from_address, to_address, amount):
        self.from_address = from_address
        self.to_address = to_address
        self.amount = amount
        
    def calculate_hash(self):
        hash_str = self.from_address + self.to_address + str(self.amount)
        return sha256(hash_str.encode('utf-8')).hexdigest()
        
    def sign_transaction(self, signing_key):
        signing_key = SigningKey.from_string(bytearray.fromhex(signing_key), curve=SECP256k1)
        if signing_key.get_verifying_key().to_string().hex() != self.from_address:
            raise ValueError('you cannot sign this transaction!')
        hash_tr = self.calculate_hash()
        self.signature = signing_key.sign(hash_tr.encode('utf-8')).hex()
        
    def is_valid(self):
        if self.from_address == None:
            return True
        if self.signature == False or len(self.signature)==0:
            raise ValueError('Missing signature!')
        public_key = VerifyingKey.from_string(bytearray.fromhex(self.from_address), curve=SECP256k1)
        try:
            return public_key.verify(bytes.fromhex(self.signature), self.calculate_hash().encode('utf-8'))
        except Exception as e:
            print(e)
            return False
        

class Block:
    def __init__(self, timestamp, transactions, previous_hash=''):
        self.timestamp = timestamp
        self.transactions = transactions
        self.count = 0
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()
        
    def calculate_hash(self):
        if self.previous_hash != "0":
            temp_transactions = [trans.__dict__ for trans in self.transactions]
        else:
            temp_transactions = self.transactions
        hash_str = self.timestamp + json.dumps(temp_transactions) + self.previous_hash + str(self.count)
        return sha256(hash_str.encode('utf-8')).hexdigest()
    
    def mining_block(self, difficulty):
        while self.hash[0:difficulty] != '0'*difficulty:
            self.count += 1
            self.hash = self.calculate_hash()       
        #print(f'block mined!')
        
    def has_valid_transactions(self):
        for trans in self.__dict__['transactions']:
            if trans.is_valid()==False:
                return False
        return True
    
    def content(self):
        msg = self.__dict__.copy()
        msg_transactions = []
        for trans in msg['transactions']:
            if self.previous_hash != '0':
                msg_transactions.append(trans.__dict__)
        msg['transactions'] = msg_transactions
        print(json.dumps(msg, indent=2, default=str))
        print('----------------------------------------------')
    
    
class BlockChain:
    def __init__(self):
        self.chain = [self.generate_genesis_block()]
        self.pending_transactions = []
        self.reward = 100
        self.difficulty = 3
        
    def generate_genesis_block(self):
        return Block('23/04/2021', 'Genesis Block', "0")
    
    #def add_block(self, new_block):
    #    new_block.previous_hash = self.chain[-1].hash
    #    #new_block.hash = new_block.calculate_hash()
    #    new_block.mining_block(self.difficulty)
    #    self.chain.append(new_block)
        
    def mining_pending_transactions(self, mining_reward_address): 
        block = Block(datetime.now().strftime("%d/%m/%Y"), self.pending_transactions)
        self.pending_transactions.append(Transaction(None, mining_reward_address, self.reward))
        block.previous_hash = self.chain[-1].hash
        block.mining_block(self.difficulty)
        print('blocked mined!')
        self.chain.append(block)
        self.pending_transactions = []
        
    def add_transactions(self, transaction):
        if VerifyingKey.from_string(bytearray.fromhex(transaction.from_address), curve=SECP256k1) is None or VerifyingKey.from_string(bytearray.fromhex(transaction.to_address), curve=SECP256k1) is None:
            raise ValueError('from address or to address are in the wrong format')
        if transaction.is_valid()==False:
            raise ValueError('you cannot add an invalid transaction to the block!')
        self.pending_transactions.append(transaction)
        
    def get_balance(self, address):
        balace = 0
        for block in self.chain[1:]:
            for trans in block.transactions:
                trans = trans.__dict__
                if trans['from_address'] == address:
                    balace -= trans['amount']
                elif trans['to_address'] == address:
                    balace += trans ['amount']
        return balace
        
    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            if i==0:
                current_block = self.chain[i]
                if current_block.previous_hash != "0":
                    raise ValueError('previous hash of first block is not 0!!!')
            else:
                current_block = self.chain[i]
                previous_block = self.chain[i-1]
                if current_block.has_valid_transactions()==False:
                    self.chain = pickle.load(open('chain', 'rb'))
                    raise ValueError(f'current block {current_block.hash[0:8]} is not a valid transaction!')
                if current_block.hash != current_block.calculate_hash():
                    self.chain = pickle.load(open('chain', 'rb'))
                    raise ValueError(f'current_block.hash != current_block.calculate_hash()')
                elif current_block.previous_hash != previous_block.hash:
                    self.chain = pickle.load(open('chain', 'rb'))
                    raise ValueError(f' current_block.previous_hash != previous_block.hash')
        pickle.dump(self.chain, open('chain','wb'))
        print('valid blockchain')
        
    def content(self):
        for block in self.chain:
            block.content()

In [125]:
fede_chain = BlockChain()

In [126]:
private_key_marco = SigningKey.generate(SECP256k1)
public_key_marco = private_key_marco.get_verifying_key()

private_key_marco = private_key_marco.to_string().hex()
public_key_marco = public_key_marco.to_string().hex()

private_key_polo = SigningKey.generate(SECP256k1)
public_key_polo = private_key_polo.get_verifying_key()

private_key_polo = private_key_polo.to_string().hex()
public_key_polo = public_key_polo.to_string().hex()

private_key_mario = SigningKey.generate(SECP256k1)
public_key_mario = private_key_mario.get_verifying_key()

private_key_mario = private_key_mario.to_string().hex()
public_key_mario = public_key_mario.to_string().hex()

In [127]:
tx1 = Transaction(public_key_marco, public_key_polo, 100)
tx1.sign_transaction(private_key_marco)
fede_chain.add_transactions(tx1)

tx2 = Transaction(public_key_polo, public_key_marco, 30)
tx2.sign_transaction(private_key_polo)
fede_chain.add_transactions(tx2)

fede_chain.mining_pending_transactions(public_key_mario)

blocked mined!


In [128]:
fede_chain.content()

{
  "timestamp": "23/04/2021",
  "transactions": [],
  "count": 0,
  "previous_hash": "0",
  "hash": "f5fdb3e08b574038f23198910ba57061143daeabbf51b13baa17ec33e3295125"
}
----------------------------------------------
{
  "timestamp": "25/04/2021",
  "transactions": [
    {
      "from_address": "79e4e8bc934c36c59ae8b0a4fa13b2da6ee52825f018943c958352d47108555d4c56aa306ac6fa571c13f52c3be59c71e2852dce73e785a310b6184f303e20a5",
      "to_address": "6ff5ca1d3caaea0eb8aa17eaf5c78039bf98acf3b05e0a1d27a3531c9e7e701788f77b56d6bea5922dacb8a841132ad1c22200878dd99660ad5217623dbcbb48",
      "amount": 100,
      "signature": "2877a5bddafea9fd5488facdd16e5a5b38aefceb873fd5e3d965d73226342f9fe3336ce76e12ab9950f89293f92d9e5bb2764964b7a6d3f1df43292e8014f364"
    },
    {
      "from_address": "6ff5ca1d3caaea0eb8aa17eaf5c78039bf98acf3b05e0a1d27a3531c9e7e701788f77b56d6bea5922dacb8a841132ad1c22200878dd99660ad5217623dbcbb48",
      "to_address": "79e4e8bc934c36c59ae8b0a4fa13b2da6ee52825f018943c958352d471085

In [129]:
fede_chain.get_balance(public_key_marco)

-70

In [133]:
fede_chain.chain[1].transactions[0].amount = 200

In [136]:
fede_chain.is_chain_valid()

valid blockchain


In [135]:
fede_chain.content()

{
  "timestamp": "23/04/2021",
  "transactions": [],
  "count": 0,
  "previous_hash": "0",
  "hash": "f5fdb3e08b574038f23198910ba57061143daeabbf51b13baa17ec33e3295125"
}
----------------------------------------------
{
  "timestamp": "25/04/2021",
  "transactions": [
    {
      "from_address": "79e4e8bc934c36c59ae8b0a4fa13b2da6ee52825f018943c958352d47108555d4c56aa306ac6fa571c13f52c3be59c71e2852dce73e785a310b6184f303e20a5",
      "to_address": "6ff5ca1d3caaea0eb8aa17eaf5c78039bf98acf3b05e0a1d27a3531c9e7e701788f77b56d6bea5922dacb8a841132ad1c22200878dd99660ad5217623dbcbb48",
      "amount": 100,
      "signature": "2877a5bddafea9fd5488facdd16e5a5b38aefceb873fd5e3d965d73226342f9fe3336ce76e12ab9950f89293f92d9e5bb2764964b7a6d3f1df43292e8014f364"
    },
    {
      "from_address": "6ff5ca1d3caaea0eb8aa17eaf5c78039bf98acf3b05e0a1d27a3531c9e7e701788f77b56d6bea5922dacb8a841132ad1c22200878dd99660ad5217623dbcbb48",
      "to_address": "79e4e8bc934c36c59ae8b0a4fa13b2da6ee52825f018943c958352d471085